In [1]:
import numpy as np 
import pandas as pd


/kaggle/input/patient-ratings-identifying-best-drugs/Drug Reviews (Drugs.com)/drugsComTest_raw.csv
/kaggle/input/patient-ratings-identifying-best-drugs/Drug Reviews (Drugs.com)/drugsComTrain_raw.csv
/kaggle/input/patient-ratings-identifying-best-drugs/Drug Reviews (Druglib.com)/drugLibTest_raw.csv
/kaggle/input/patient-ratings-identifying-best-drugs/Drug Reviews (Druglib.com)/drugLibTrain_raw.csv


In [2]:
!pip install wolta

# Data Init and First Look

first is first, i will work only with **drugs.com** data.

In [3]:
com_train = pd.read_csv('../drugsComTrain_raw.csv')

In [4]:
com_train.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37


In [5]:
com_train.shape

(161297, 7)

In [6]:
from wolta.data_tools import seek_null

seeked = seek_null(com_train, print_columns=True)

condition has 899 null values


In [7]:
if None in list(com_train['condition'].values):
    print('None here!')
    
if np.nan in list(com_train['condition'].values):
    print('nan here!')

nan here!


In [8]:
from wolta.data_tools import col_types

types = col_types(com_train, print_columns=True)

Unnamed: 0: int64
drugName: str
condition: str
review: str
rating: float64
date: str
usefulCount: int64


In [9]:
from wolta.data_tools import unique_amounts

unique_amounts(com_train)

{'Unnamed: 0': 161297,
 'drugName': 3436,
 'condition': 885,
 'review': 112329,
 'rating': 10,
 'date': 3579,
 'usefulCount': 389}

# Data Manipulation

before everything, we will **replace** null data.

In [10]:
com_train = com_train.replace({np.nan: 'NOT POINTED'})

| feature | action | reason |
| --- | --- | --- |
| Unnamed: 0 | delete | id |
| drugName | make numerics | categorical |
| condition | make numerics | categorical |
| review | make numerics | string data |
| rating | combine with usefulCount | regression |
| date | delete | manipulative |
| usefulCount | combine with rating | regression |

In [11]:
del com_train['Unnamed: 0']
del com_train['date']

In [12]:
from wolta.data_tools import make_numerics

com_train['drugName'], name_hashed = make_numerics(com_train['drugName'], space_requested=True)
com_train['condition'], condition_hashed = make_numerics(com_train['condition'], space_requested=True)

In [13]:
for i in range(com_train.shape[0]):
    com_train['review'].values[i] = len(com_train['review'].values[i]) 

In [14]:
vals = []

for i in range(com_train.shape[0]):
    vals.append(com_train['rating'].values[i] * com_train['usefulCount'].values[i])

com_train['usefulness'] = np.array(vals)

del com_train['rating']
del com_train['usefulCount']

# Train Data Preparation

In [15]:
y_train = com_train['usefulness'].values
del com_train['usefulness']
X_train = com_train.values
del com_train

# Test Data Preparation

In [16]:
df = pd.read_csv('../drugsComTest_raw.csv')

In [17]:
df.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,163740,Mirtazapine,Depression,"""I&#039;ve tried a few antidepressants over th...",10.0,"February 28, 2012",22
1,206473,Mesalamine,"Crohn's Disease, Maintenance","""My son has Crohn&#039;s disease and has done ...",8.0,"May 17, 2009",17
2,159672,Bactrim,Urinary Tract Infection,"""Quick reduction of symptoms""",9.0,"September 29, 2017",3
3,39293,Contrave,Weight Loss,"""Contrave combines drugs that were used for al...",9.0,"March 5, 2017",35
4,97768,Cyclafem 1 / 35,Birth Control,"""I have been on this birth control for one cyc...",9.0,"October 22, 2015",4


In [18]:
del df['Unnamed: 0']
del df['date']

In [19]:
df['drugName'] = df['drugName'].replace(name_hashed)
df['condition'] = df['condition'].replace(condition_hashed)

In [20]:
new_vals = []
latest = -1
for obj in list(df['drugName'].unique()):
    if str(type(obj)).__contains__('str'):
        new_vals.append(obj)
    elif obj > latest:
        latest = obj

new_dict = {}
latest += 1

for val in new_vals:
    new_dict[val] = latest
    latest += 1

df['drugName'] = df['drugName'].replace(new_dict)

/tmp/ipykernel_18/3980758631.py:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['drugName'] = df['drugName'].replace(new_dict)


In [21]:
new_vals = []
latest = -1
for obj in list(df['condition'].unique()):
    if str(type(obj)).__contains__('str'):
        new_vals.append(obj)
    elif obj > latest:
        latest = obj

new_dict = {}
latest += 1

for val in new_vals:
    new_dict[val] = latest
    latest += 1

new_dict[np.nan] = latest

df['condition'] = df['condition'].replace(new_dict)

/tmp/ipykernel_18/1736482287.py:18: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['condition'] = df['condition'].replace(new_dict)


In [22]:
for i in range(df.shape[0]):
    df['review'].values[i] = len(df['review'].values[i]) 

In [23]:
vals = []

for i in range(df.shape[0]):
    vals.append(df['rating'].values[i] * df['usefulCount'].values[i])

df['usefulness'] = np.array(vals)

del df['rating']
del df['usefulCount']

In [24]:
y_test = df['usefulness'].values
del df['usefulness']
X_test = df.values
del df

# Model

In [25]:
from wolta.model_tools import compare_models

compare_models('reg',
              ['cat', 'raf', 'dtr', 'lin', 'ext'],
              ['sq'],
              X_train, y_train, X_test, y_test)

CatBoost
Mean Squared Error: 91206.9938556314
***
Random Forest
Mean Squared Error: 108591.45621444688
***
Decision Tree
Mean Squared Error: 162050.68604262712
***
Linear Regression
Mean Squared Error: 115089.8611696575
***
Extra Tree
Mean Squared Error: 164456.47597405757
***


In [26]:
from catboost import CatBoostRegressor

model = CatBoostRegressor(verbose=False)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [27]:
avg = 0

for i in range(y_pred.shape[0]):
    avg += np.absolute(y_pred[i] - y_test[i])

avg /= y_pred.shape[0]
print(avg)

179.38990017060115


In [28]:
print(y_test[:10])
print(y_pred[:10])

[220. 136.  27. 315.  36.  52.   6. 288. 147.   6.]
[393.98388635 262.22645341 220.03295692 193.38436428 -16.53690904
 288.83385053  59.89583172 277.96804132 331.16987271  52.92404199]
